In [98]:
import pandas as pd
from process import run_ols
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from scipy.stats import norm
from MLE import MLE
from process import run_ols

In [99]:
players = [] # array of dataframes where players[i] is the i+1th player's data
for i in range(1000):
    file = "../data/player_data_one_hot/student_" + str(i + 1) + "_one_hot.csv"
    players.append(pd.read_csv(file))
# maybe find trend between all player's m, n, e, find which covariates affect this sport and
# make a model given expected score and covariates, obtain m, n, and e?

In [100]:
# Step 1: obtain all data from singular_one_hot
soh = pd.read_csv("../data/singular_one_hot.csv")
soh = soh.drop(columns=["previous_sport"])
soh2 = soh.to_numpy().T
np. set_printoptions(threshold=np.inf, suppress=True, linewidth=np. inf)
print(np.cov(soh2))
morning_oh = soh[soh["x0_morning"] == 1.0]
noon_oh = soh[soh["x0_noon"] == 1.0]
evening_oh = soh[soh["x0_evening"] == 1.0]

[[83341.58415842 83341.58415842 83341.58415842   171.23358386     2.76511051     3.23776828    24.00440044    15.71962196    -0.50605061            nan    50.4050405     -2.45524552    -2.42524252     4.88048805     5.3130313     -5.3130313     -5.51210121     1.90189019     4.97344734    -1.36323632]
 [83341.58415842 83341.58415842 83341.58415842   171.23358386     2.76511051     3.23776828    24.00440044    15.71962196    -0.50605061            nan    50.4050405     -2.45524552    -2.42524252     4.88048805     5.3130313     -5.3130313     -5.51210121     1.90189019     4.97344734    -1.36323632]
 [83341.58415842 83341.58415842 83341.58415842   171.23358386     2.76511051     3.23776828    24.00440044    15.71962196    -0.50605061            nan    50.4050405     -2.45524552    -2.42524252     4.88048805     5.3130313     -5.3130313     -5.51210121     1.90189019     4.97344734    -1.36323632]
 [  171.23358386   171.23358386   171.23358386   410.95126972     4.74025896     4.06617735

In [101]:
# Step 2: run regression on the data for morning
formula = "score ~ fitness_score + extra_strategy_sessions_attended + hours_of_sleep + meals + east_coast + early_bird + frosh + junior + senior"
morning_result = run_ols(formula, morning_oh, print_data=True)

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.514
Model:                            OLS   Adj. R-squared:                  0.505
Method:                 Least Squares   F-statistic:                     61.02
Date:                Mon, 20 Mar 2023   Prob (F-statistic):           1.00e-75
Time:                        21:24:29   Log-Likelihood:                -2152.5
No. Observations:                 530   AIC:                             4325.
Df Residuals:                     520   BIC:                             4368.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [102]:
# Step 3: run regression on the data for noon
formula = "score ~ fitness_score + extra_strategy_sessions_attended + hours_of_sleep + meals + east_coast + early_bird + frosh + junior + senior"
noon_result = run_ols(formula, noon_oh, print_data=True)

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     295.4
Date:                Mon, 20 Mar 2023   Prob (F-statistic):               0.00
Time:                        21:24:30   Log-Likelihood:                -13236.
No. Observations:                3210   AIC:                         2.649e+04
Df Residuals:                    3200   BIC:                         2.655e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [103]:
# Step 4: run regression on the data for evening
formula = "score ~ fitness_score + extra_strategy_sessions_attended + hours_of_sleep + meals + east_coast + early_bird + frosh + junior + senior"
evening_result = run_ols(formula, evening_oh, print_data=True)

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.597
Model:                            OLS   Adj. R-squared:                  0.594
Method:                 Least Squares   F-statistic:                     205.8
Date:                Mon, 20 Mar 2023   Prob (F-statistic):          1.72e-239
Time:                        21:24:30   Log-Likelihood:                -4987.2
No. Observations:                1260   AIC:                             9994.
Df Residuals:                    1250   BIC:                         1.005e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [104]:
# Step 5: prediction function
keys = {"fitness_score", "extra_strategy_sessions_attended", "hours_of_sleep", "meals", "east_coast", "early_bird", "frosh", "junior", "senior"}
def predict_score(type, player):
    ols_prediction_model = None
    player_avg = player.mean(axis=0)
    if type == "morning":
        ols_prediction_model = morning_result.params
    elif type == "noon":
        ols_prediction_model = noon_result.params
    elif type == "evening":
        ols_prediction_model = evening_result.params
    else:
        print("this is illegal")

    # return prediction
    return ols_prediction_model["Intercept"] + np.sum([ols_prediction_model[key] * player_avg[key] for key in keys])

In [105]:
pred_score = np.zeros((3, 1000))

for i in range(1000):
    player = players[i]
    # replaced by predict_score function
    pred_score[0][i] = predict_score("morning", player)
    pred_score[1][i] = predict_score("noon", player)
    pred_score[2][i] = predict_score("evening", player)
    # for _, row in player.iterrows():
    #     if row["game_scheduled"] == "morning":
    #         total_score[0][i] += row["score"]
    #         num_score[0][i] += 1
    #     elif row["game_scheduled"] == "noon":
    #         total_score[1][i] += row["score"]
    #         num_score[1][i] += 1
    #     elif row["game_scheduled"] == "evening":
    #         total_score[2][i] += row["score"]
    #         num_score[2][i] += 1

# avg_score = np.divide(total_score, num_score, out=np.zeros_like(total_score), where=num_score!=0)
# pred_score
# avg_score[0] = "m", avg_score[1] = "n", avg_score[2] = "e"
# huge problem with expected_score as some of the scores are 0
# maybe find trend between all player's m, n, e, find which covariates affect this sport and make a model given expected score and covariates, obtain m, n, and e?

/var/folders/h3/v6p39mmn4snd_1b2tm2knxnw0000gn/T/ipykernel_65668/185969212.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  player_avg = player.mean(axis=0)


In [106]:
expected_score = np.sum(pred_score, axis=0) / 3
# determine f(x, y)
mle = MLE("../data/previous_season_results.csv")
sigma = mle.calculate_MLE() # obtain from MLE
print(sigma)

def winProb(score_1, score_2):
    return 1 - norm(loc = score_1 - score_2 , scale = sigma).cdf(0)

11.000027816393704


In [107]:
# test top scores
player_data = []
for i in range(1000):
    player_data.append(pd.read_csv("../data/player_data/student_" + str(i + 1) + ".csv"))

player_avg = []
for i in range(1000):
    player_avg.append(np.average(player_data[i]["score"]))

player_avg = np.array(player_avg)

top = np.argpartition(player_avg, -10)[-10:] + 1
print("these are our top athletes:", top)

# team score
top_s = player_avg[top - 1]
tts = np.average(top_s)
print("our team score is: ", tts)

# expected number of wins
matchup = pd.read_csv("../data/season_match_up.csv")
matchup = matchup.to_numpy()[0, 1:] # 20 games
E_win = np.sum([winProb(tts, matchup_score) for matchup_score in matchup])
print("Expected number of wins: ", E_win)

these are our top athletes: [138 235 461 660 359 129 791 521 760 268]
our team score is:  76.8493
Expected number of wins:  11.948384818487417


In [108]:
# to maximize E[win], we should choose the 10 highest players
top_athletes = np.argpartition(expected_score, -10)[-10:] + 1
print("these are our top athletes:", top_athletes)

# team score
top_scores = expected_score[top_athletes - 1]
top_team_score = np.average(top_scores)
print("our team score is: ", top_team_score)

# expected number of wins
matchup = pd.read_csv("../data/season_match_up.csv")
matchup = matchup.to_numpy()[0, 1:] # 20 games
E_win = np.sum([winProb(top_team_score, matchup_score) for matchup_score in matchup])
print("Expected number of wins: ", E_win)

these are our top athletes: [856 280 760 725 295 888 842 129 976 608]
our team score is:  66.89204295706729
Expected number of wins:  7.8269666841508965


In [109]:
consistency = np.zeros((3, 1000))
for i in range(consistency.shape[0]):
    for j in range(consistency.shape[1]):
        consistency[i, j] = np.sum([winProb(pred_score[i, j], matchup_score) for matchup_score in matchup])

# (3, 1000)
consistency = np.average(consistency, axis=0)

In [152]:
consistent_athletes = np.argpartition(consistency, -10)[-10:] + 1
print("these are our consistent athlete: ", consistent_athletes)

# team score
consistent_scores = expected_score[consistent_athletes - 1]
consistent_team_score = np.average(consistent_scores)
consistent_morning_score = np.average(pred_score[0, consistent_athletes - 1])
consistent_noon_score = np.average(pred_score[1, consistent_athletes - 1])
consistent_evening_score = np.average(pred_score[2, consistent_athletes - 1])
print("our team score is: ", consistent_team_score, consistent_morning_score, consistent_noon_score, consistent_evening_score)

# probability of losing a single game
p_lose = np.prod([(winProb(consistent_morning_score, matchup_score) + winProb(consistent_noon_score, matchup_score) + winProb(consistent_evening_score, matchup_score)) / 3 for matchup_score in matchup])
p_lose

these are our consistent athlete:  [856 280 760 725 129 295 842 976 888 608]
our team score is:  66.89204295706729 64.74517520175291 62.151500699902854 73.77945296954611


2.9275469999615842e-15

In [156]:
used_athletes = set(list(consistent_athletes - 1) + list(top - 1))
print("these are our used athletes:", used_athletes)
pred_score.shape
# west coast morning, east coast evening
# np.argpartition(expected_score, -10)[-10:] + 1
top_of_the_morning = np.argpartition(pred_score[0], -10 - len(used_athletes))[-10 - len(used_athletes):] + 1
top_of_the_evening = np.argpartition(pred_score[2], -10 - len(used_athletes))[-10 - len(used_athletes):] + 1

best_western = []
eastern_babies = []
for morning_athlete in top_of_the_morning:
    if morning_athlete not in used_athletes:
        best_western.append(morning_athlete)

    if len(best_western) == 10:
        break

for evening_athlete in top_of_the_evening:
    if evening_athlete not in used_athletes and evening_athlete not in best_western:
        eastern_babies.append(evening_athlete)

    if len(eastern_babies) == 10:
        break

print("western team: ", best_western)
print("eastern Team: ", eastern_babies)

these are our used athletes: {128, 294, 358, 520, 841, 137, 234, 460, 267, 855, 975, 887, 659, 724, 790, 759, 279, 607}
western team:  [526, 761, 856, 683, 725, 836, 942, 877, 349, 605]
eastern Team:  [415, 608, 976, 295, 842, 280, 629, 521, 268, 190]
